<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Notebooks/Llama_prototyp_r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Llama Prototype

SUMA 08/09/2023

#Instructions

Create Folder for the experiment on personal Google Drive
1.   Get model from: https://huggingface.co/eachadea/legacy-ggml-vicuna-13b-4bit/tree/main
2.   Create project folder
3.   Create Folder 'model' in project folder  
4.   Save model in folder 'model'
5.   Create folder 'data' in project folder
6.   Save folder CrisiLexT26 in folder 'data'
3.   Mount Google Drive
4.   Update code with Paths
5.   Pip install llama-cpp-python package version 0.1.4.9
6.   Load packages llama_ccp and json





In [1]:
# Inastalling llama
# !pip install llama-cpp-python
# !pip install llama-cpp-python==0.1.78
!pip install llama-cpp-python==0.1.49

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.49-cp310-cp310-linux_x86_64.whl size=181338 sha256=ac54d4485d887a18caf856f9c3109e57b0396eb4e19e33954771ceac3e5eaea8
  Stored in directory: /root/.cache/pip/wheels/01/f3/f7/b070cbde244babc2ead31326c399bd2d4187ffffed9ea09ce9
Successfully built llama-cpp-python


In [2]:
# Import Packages
from llama_cpp import Llama
import pandas as pd
import numpy as np
import json
import os
import csv


In [3]:
# Mounting GDrive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
# # Reading the CSV file
# file_path_id = "/content/gdrive/MyDrive/iLab2/data/CrisisLexT26/2013_Australia_bushfire/2013_Australia_bushfire-tweetids_entire_period.csv"
file_path = "/content/gdrive/MyDrive/iLab2/data/CrisisLexT26/2013_Australia_bushfire/2013_Australia_bushfire-tweets_labeled.csv"

# AsustBF_2013_id = pd.read_csv(file_path_id)
AsustBF_2013 = pd.read_csv(file_path)

# Displaying the first few rows of the DataFrame
# print(AsustBF_2013_id.head())
# print(AsustBF_2013.head())
AsustBF_2013.head()


,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness
0,388923108922781697,"It's ridiculous,,,RT @ThatGuyTiisetso: #redoct...",Not labeled,Not labeled,Not related
1,388929052247347200,A Total Fire Ban has been declared for the Gre...,Media,Caution and advice,Related and informative
2,388945930130841600,NSW National Parks &amp; Wildlife Service have...,Government,Caution and advice,Related and informative
3,388955358901067776,Fire Danger Update - Tomorrow signs will be on...,Government,Caution and advice,Related and informative
4,388970580026089472,This #RedOctober shit is just the beginning of...,Not labeled,Not labeled,Not related


In [5]:
# Select 5 random samples from 'AsustBF_2013' and create a new DataFrame 'working_df'
working_df = AsustBF_2013.sample(n=5, random_state=42)

# Rename the columns
new_column_names = ['id', 'tweet', 'source', 'type', 'info']
working_df.columns = new_column_names

# Create a new column to store sentiment scores
working_df['sentiment_score'] = 0

working_df.head(10)


,id,tweet,source,type,info,sentiment_score
1177,395336145174675456,RT @702sydney: Image of @LindaMottram w/ Gener...,Outsiders,Affected individuals,Related and informative,0
864,392792824086675456,Nothing lower than fire looters: NSW cops http...,Outsiders,Other Useful Information,Related and informative,0
101,390633776776105984,RT @NSWRFS: #Lithgow Fire: The fire has jumped...,Government,Other Useful Information,Related and informative,0
439,391348398085124096,RT @CraigHavenr: Man that lost his house in NS...,Outsiders,Infrastructure and utilities,Related and informative,0
58,389803627541704705,RT @King_Jones5: PSA: LAMAR UNIVERSITY!!! YOU...,Not labeled,Not labeled,Not related,0


In [6]:
# Load Model
print("Load Model ...")
llm = Llama(model_path = '/content/gdrive/MyDrive/iLab2/model/ggml-vicuna-13b-4bit-rev1.bin', verbose=True )
print("Load Model")


Load Model ...
Load Model


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [14]:
# Running Model over data
for index, row in working_df.iterrows():

    # Construct the question with the text snippet
    question_construct = "Can you give me the sentiment for the following snippet: " + row['tweet'] + "? Answer:"

    # Run the model to analyze sentiment
    output = llm(question_construct, max_tokens=100, stop=["\n", "Question:", "Q:"], echo=True)

    # Extract the sentiment score from the output
    sentiment_score = output.get("choices")[0].get("text")[-8:]

    # Update the 'sentiment_score' column in the DataFrame
    working_df.at[index, 'sentiment_score'] = sentiment_score


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [15]:
# Print output
working_df.head(10)


,id,tweet,source,type,info,sentiment_score
1177,395336145174675456,RT @702sydney: Image of @LindaMottram w/ Gener...,Outsiders,Affected individuals,Related and informative,snippet.
864,392792824086675456,Nothing lower than fire looters: NSW cops http...,Outsiders,Other Useful Information,Related and informative,Negative
101,390633776776105984,RT @NSWRFS: #Lithgow Fire: The fire has jumped...,Government,Other Useful Information,Related and informative,Neutral
439,391348398085124096,RT @CraigHavenr: Man that lost his house in NS...,Outsiders,Infrastructure and utilities,Related and informative,Negative
58,389803627541704705,RT @King_Jones5: PSA: LAMAR UNIVERSITY!!! YOU...,Not labeled,Not labeled,Not related,Positive


In [11]:
# Assuming 'output' is a dictionary containing the JSON data
output = {
  "id": "cmpl-aeff8f79-3071-4fa6-bacf-c44a31d45b9b",
  "object": "text_completion",
  "created": 1694298540,
  "model": "/content/gdrive/MyDrive/iLab2/model/ggml-vicuna-13b-4bit-rev1.bin",
  "choices": [
    {
      "text": "Question: Can you give me the sentiment for the following snippet: 'Run a marathon in under two hours. Impossible? Not for Nike (@Nike). Last May, the company brought three of the best runners on the planet together in Italy to set a new record in a closed-door marathon that was broadcast live on Twitter.'? Answer: Positive",
      "index": 0,
      "logprobs": "bla",
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 3,
    "total_tokens": 78
  }
}

# Access the value associated with the key "text"
sentiment_score = output.get("choices")[0].get("text")

last_8_characters = sentiment_score[-8:]

# Print the last 8 characters
print(last_8_characters)

# Print the value
print(sentiment_score)


Positive
Question: Can you give me the sentiment for the following snippet: 'Run a marathon in under two hours. Impossible? Not for Nike (@Nike). Last May, the company brought three of the best runners on the planet together in Italy to set a new record in a closed-door marathon that was broadcast live on Twitter.'? Answer: Positive
